In [4]:
import osmnx as ox
import networkx as nx
import random
import time

def get_period_dependent_random_float(t_seconds: float, period_hours: float) -> float:
    """
    Generates a deterministic random float that changes only after a specified period.

    Args:
        t_seconds: The current simulation time in seconds.
        period_hours: The duration for which the random state should remain constant, in hours.

    Returns:
        A float between 0.0 and 1.0.
    """
    # 1. Convert the period from hours to seconds
    period_seconds = period_hours * 3600
    
    # 2. Calculate the seed using integer division. This is the key step.
    # The seed will be the same for every `t_seconds` within the same period.
    if period_seconds <= 0:
        # Handle edge case to avoid division by zero
        seed_value = int(t_seconds)
    else:
        seed_value = int(t_seconds // period_seconds)
    
    # 3. Set the seed and generate the number
    random.seed(seed_value)
    
    return random.random()

start = time.time()

print(get_period_dependent_random_float(start, 0.5))

0.3904094940995838


In [ ]:

def add_ev_simulation_attributes(network: nx.MultiDiGraph) -> nx.MultiDiGraph:
    """
    Adds custom attributes to each edge for EV power consumption simulation.

    This function adds:
    - congestion: A random float simulating traffic.
    - is_open: A random integer (1 or 0) simulating road closures.
    - power_adjustment_factor: A derived value based on grade and congestion.
    """
    print("Adding custom EV simulation attributes (congestion, closures, etc.)...")

    # This is a simplified model to demonstrate the concept.
    # In a real-world scenario, these values would come from real-time data or more complex models.
    for u, v, key, data in network.edges(keys=True, data=True):
        # 1. Add Congestion: Random float between 0.0 (clear) and 0.8 (heavy traffic)
        congestion = round(random.uniform(0.0, 0.8), 4)
        network.edges[u, v, key]['congestion'] = congestion

        # 2. Add Open/Closed Status: 95% chance of being open (1), 5% chance of being closed (0)
        is_open = random.choices([1, 0], weights=[95, 5], k=1)[0]
        network.edges[u, v, key]['is_open'] = is_open

        # 3. Calculate a Power Adjustment Factor (using 'grade' and 'congestion')
        # The 'grade' attribute should be added before calling this function.
        grade = data.get('grade', 0.0) # Grade is the slope (e.g., 0.03 for 3%)
        
        # Start with a baseline factor of 1.0
        power_factor = 1.0
        
        # Increase power cost significantly for uphill travel
        if grade > 0.01: # Steeper than 1%
            power_factor += grade * 15 # The steeper the hill, the higher the cost
        
        # Decrease power cost for downhill travel (regenerative braking)
        # Note: Regeneration is not 100% efficient.
        elif grade < -0.01:
            power_factor -= abs(grade) * 7 # Recover some energy, but less than the cost to go up
            
        # Increase power cost for congestion (stop-and-go traffic is inefficient)
        power_factor *= (1 + congestion * 0.5) # Add up to 40% more cost for heavy traffic
        
        # Ensure the factor doesn't go below a certain threshold (e.g., cannot generate infinite power)
        network.edges[u, v, key]['power_adjustment_factor'] = max(0.2, round(power_factor, 4))
        
    print(" Custom EV attributes added.")
    return network



In [ ]:
# 1. Define a very small Bounding Box for rapid testing
north, south, east, west = 40.6541, 40.5921, -74.0437, -73.8885

print(f" Downloading road network for TINY bounding box: ({north}, {south}, {east}, {west})")

try:
    # 2. Download the road network
    road_network = ox.graph_from_bbox(bbox=(north, south, east, west), network_type='drive', simplify=True,retain_all=True)
    print(f"Download complete with {len(road_network.nodes)} nodes and {len(road_network.edges)} edges.")
    
    # --- Attribute Addition Pipeline ---
    
    # 3. Add Elevation and Grade (Slope) - CRITICAL for EV calculations
    # This adds 'elevation', 'grade', and 'grade_abs' to nodes and edges
    print("Adding elevation and grade data from OpenStreetMap...")
    road_network = ox.add_node_elevations_raster(road_network, "srtm_30m.tif")
    road_network = ox.add_edge_grades(road_network)
    print(" Grade data added.")

    # 4. Add Speed and Travel Time (standard attributes)
    print("Adding standard speed and travel time attributes...")
    road_network = ox.add_edge_speeds(road_network)
    road_network = ox.add_edge_travel_times(road_network)
    print(" Speed and travel time added.")
    
    # 5. Add our new custom EV attributes
    road_network_with_attributes = add_ev_simulation_attributes(road_network)
    
    # 6. Verification Step: Inspect an edge to see all the new attributes
    print("\n Verifying all attributes on a sample edge...")
    
    if not road_network_with_attributes.edges:
        print("   The graph has no edges to verify.")
    else:
        sample_edge = list(road_network_with_attributes.edges(keys=True, data=True))[0]
        u, v, key, data = sample_edge
        
        print(f"   Sample Edge (from node {u} to {v}):")
        print("-" * 35)
        # Standard Attributes
        print(f"     - Road Type ('highway'): {data.get('highway', 'N/A')}")
        print(f"     - Length (meters):       {data.get('length', 0):.2f}")
        print(f"     - Speed (km/h):          {data.get('speed_kph', 0):.2f}")
        print(f"     - Travel Time (sec):     {data.get('travel_time', 0):.2f}")
        # EV-Specific Attributes
        print(f"     - Grade (slope):         {data.get('grade', 0):.4f}")
        print(f"     - Congestion:            {data.get('congestion', 'N/A')}")
        print(f"     - Is Open:               {data.get('is_open', 'N/A')}")
        print(f"     - Power Adj. Factor:     {data.get('power_adjustment_factor', 'N/A')}")
        print("-" * 35)

except Exception as e:
    print(f"\n An error occurred: {e}")
    print("   This might be due to an API issue, missing elevation data, or no network in the tiny box.")